# 4. 신경망 학습
- 학습이란 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것을 뜻한다.
- 이 장에서는 신경망이 학습할 수 있도록 해주는 지표인 손실 함수를 소개한다.

## 4.1 데이터에서 학습한다.

### 4.1.1 데이터 주도 학습
- 어떠한 이미지를 분류할 때, 
- 1. 사람은 이미지의 특징을 보고 규칙을 만들어서 판단
- 2. 컴퓨터 비전에서는 특징을 추출하고, 이 추출된 특징의 패턴을 학습
- 3. 딥러닝은 특징, 학습까지 한 번에 결과를 도출해준다. (end to end machine learning 이라고도 한다.)

### 4.1.2 훈련 데이터와 시험 데이터
- 당연한 뻘 소리를 해놓았다.
- 오버피팅을 피하는 것이 기계학습의 중요한 과제이기도 하다.

## 4.2 손실 함수

- 현재 상태의 가중치 매개변수들을 가장 좋은 가중치 매개변수의 값을 탐색해야하는데 이 지표를 신경망에서 '손실 함수' 라고 한다.
- 신경망에서는 오차제곱합과 교차 엔트로피를 사용한다.

### 4.2.1 오차제곱합
- 가장 많이 쓰이는 손실 함수는 오차제곱합 이다.

### 4.2.2 교차 엔트로피

- -sigma(Tk * log(Yk))
- T는 실제 값 배열이고(one_hot_encoding)
- Y는 예측 값 배열이다.(softmax 결과값)
- 두 개의 곱을 할 때, 어차피 one_hot_encoding으로 인해 0의 값들은 곱을 해도 0이 출력되고 예측값과 softmax 확률 값만 곱해진다.
- 이렇게 곱해진 값들을 최소화 해야한다.

### 4.2.3 미니매치 학습
- 수십, 수천만개의 빅데이터 수준이 되면 모든 데이터를 대상으로 손실 함수를 계산하고 그 총합을 구하려면 시간이 걸린다.
- 일럴 경우 데이터의 일부 표본을 뽑아 근사치로 이용한다. 이를 미니배치라고 한다.

In [40]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [41]:
x_train = x_train.reshape(60000,784)
x_test = x_test.reshape(10000,784)

x_train = x_train/255.0
x_test = x_test/255.0

In [42]:
print(x_train.shape)
print(x_test.shape)

(60000, 784)
(10000, 784)


In [43]:
# one_hot_encoding 진행
from pandas import get_dummies
y_train = np.array(get_dummies(y_train))
y_test = np.array(get_dummies(y_test))

In [45]:
print(y_train.shape)
print(y_test.shape)

(60000, 10)
(10000, 10)


In [46]:
# 10장만 빼내기
batch_size = 10
batch_mask = np.random.choice(60000,10)
print(batch_mask)
x_batch = x_train[batch_mask]
y_batch = y_train[batch_mask]

[13860 27964  1378 12657 55385 40589  5769 47182 42762    20]


### 4.2.4 (배치용) 교차 엔트로피 오차 구현하기

In [47]:
def cross_entropy_error(y,t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    
    return -np.sum(t * np.log(y + e-7)) / batch_size

### 4.2.5 왜 손실 함수를 정의하는가?
- 정확도를 지표로 삼게 되면 매개 변수의 미분이 대부분의 장소에서 0이 되기 때문이다.
- 예를들어, 100장의 데이터 중 32장을 인식한다면 32%의 정확도이다. 가중치 매개변수의 값을 조금 바꿔도 여전히 32%이고 갑자기 어느 순간 33% 로 불연속적으로 증가하게 된다.
- 손실 함수를 지표로 삼게되면 손실 함수의 값도 연속적으로 조금씩 조금씩 변화한다.

## 4.3 수치 미분

### 4.3.1 미분

- 순간 변화량을 의미한다.
- 아주 작은 차분으로 미분하는 것은 '수치 미분'이라고 한다. 오차가 포함된다.
- 수식을 전개해 해석적으로 미분하는 것(y = x^2   >>  dy/dx = 2x)은 진정한 미분값을 구해준다.

### 4.3.3 편미분
- 변수가 2개 이상일 때, 하나의 변수에 대해 미분을 진행한다.

## 4.4 기울기

- 편미분의 기울기는 모든 변수의 편미분을 벡터로 정리한 것을 말한다.
- 이 기울기 벡터가 가르키는 방향이 함수의 출력 값을 가장 크게 줄이는 방향이다.

### 4.4.1 경사하강법
- 함수의 최소값, 안장점이 되는 장소에서는 기울기가 0이다.
- 항상 최소값을 가르키는 것은 아니지만, 그 방향으로 가야 함수의 값을 줄일 수 있다.
- 현 위치에서느 기울어진 방향으로 일정 거리만큼 이동하고 나아가는 것을 반복한다. 이렇게 함수의 값을 점차 줄이는 것이 경사법이다.